# Main Code

In [13]:
import numpy as np
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F

## Get Data

In [15]:
print('loading...', flush = True)
X_train = np.load('./data/X_train.npy', allow_pickle= True)  # X_train and X_test is pickled
Y_train = np.load('./data/Y_train.npy')
X_test = np.load('./data/X_test.npy', allow_pickle= True)
Y_test = np.load('./data/Y_test.npy')
print('done')

loading...
done


## Preprocessing - converting conditions into 'possibility maps'

In [16]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((60000, 2, 8), (60000, 8, 8), (10000, 2, 8), (10000, 8, 8))

In [17]:
X_train.shape[-1]

8

In [23]:
def searchcombinationsUtil(k, n):
    """
    k: number of elements (>= 1)
    n: total sum of elements
    return all possible combinations of k numbers that add up to n, regarding its order
    """
    # Recursive function
    
    if k == 1:
        return [[n]]
    else:
        output = []
        for i in range(0, n+1):
            output += [[i]+items for items in searchcombinationsUtil(k-1, n-i)]
        return output        

In [43]:
def pixel_val_calculator(constraint, N):
    """
    constraint: the condition(constraint) of a single row/column in a nonogram
    N: the length of the width/height of the nonogram
    returns a vector with length N, and each value of the vector is the possibility of the corresponding pixel to be colored
    """
    total_colored = np.sum(constraint)

    if(len(constraint) == 0):
        return [0 for _ in range(N)]
    else:
        combinations = searchcombinationsUtil(k=int(len(constraint)+1), n= int(N-total_colored-len(constraint)+1))
        output = []

        for each_combination in combinations:
            pixel_val = []
            for idx, elements in enumerate(each_combination):
                if (idx == 0) or (idx == len(constraint)):
                    pixel_val += [0 for _ in range(elements)]
                else:
                    pixel_val += [0 for _ in range(elements+1)]
                if idx != (len(constraint)):
                    pixel_val += [1 for _ in range(constraint[idx])]
            output.append(pixel_val)
        
        output = np.array(output, dtype = np.float64)
        output = np.sum(output, axis = 0)
        output/= len(combinations)

        return output

In [44]:
pixel_val_calculator([7, 1], 10)

array([0.66666667, 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 0.33333333, 0.33333333, 0.66666667])

In [50]:
def possibility_map_generator(X):
    """
    X: (batch_size, 2, N) or (batch_size, 2, N, t) only if all constraints are composed of same number of blocks (=t)
    (2, N): each nonogram puzzle
    2 stands for each condition (row condition, column condition)
    N stands for the number of pixels (Number of total constraints)
    returns possibility_map: (batch_size, 2, N, N)
    """
    if X.ndim == 3:
        N = X.shape[-1]
    if X.ndim == 4:
        N = X.shape[-2]

    possibility_map = []
    for puzzles in X:
        row_condition = puzzles[0]
        row_map = []
        for constraints in row_condition:
            row_map.append(pixel_val_calculator(constraint=constraints, N=N))
        row_map = np.array(row_map)

        column_condition = puzzles[1]
        column_map = []
        for constraints in column_condition:
            column_map.append(pixel_val_calculator(constraint=constraints, N=N))
        column_map = np.array(column_map)
        column_map = column_map.T

        possibility_map.append(np.array([row_map, column_map]))
        
    possibilty_map = np.array(possibility_map)
    return possibility_map

In [52]:
a = np.array([[[2], [1]], [[1], [2]]])
print(a.shape)
possibility_map_generator(np.expand_dims(a,0))

(2, 2, 1)


[array([[[1. , 1. ],
         [0.5, 0.5]],
 
        [[0.5, 1. ],
         [0.5, 1. ]]])]